### First Follow

In [294]:
import os
import pandas as pd
import numpy as np

In [295]:
file_path = "lib/input/"
file_name = "test_video.txt"
# file_name = "example_1.txt"
try:
    codigo_entrada = open(file_path + file_name, "r", encoding="utf8").read()
except:
    codigo_entrada = """S -> aSbc | D
D -> dD | d"""

In [296]:
simbolo_inicial = codigo_entrada[0]

In [297]:
print(codigo_entrada)

S -> ACB | CbB | Ba
A -> da | BC
B -> g | λ
C -> h | λ



In [298]:
import re

In [299]:
qtd_linhas = codigo_entrada.count("\n") + 1
qtd_linhas

5

In [300]:
### Encontrar
regex_terminais = r"[a-z+:=*-/]"
matches = re.finditer(regex_terminais, codigo_entrada.replace("->"," "), re.MULTILINE)
variaveis = []
for match in matches:
    variaveis.append(match.group())

variaveis_terminais = list(set(variaveis))

variaveis_terminais

['a', 'h', 'g', 'd', 'b']

In [301]:
regex_nao_terminais = r"[A-Z]"
matches = re.finditer(regex_nao_terminais, codigo_entrada.replace("->"," "), re.MULTILINE)
variaveis = []
for match in matches:
    variaveis.append(match.group())

variaveis_nao_terminais = list(set(variaveis))
variaveis_nao_terminais

['A', 'B', 'C', 'S']

In [302]:
print(codigo_entrada)

S -> ACB | CbB | Ba
A -> da | BC
B -> g | λ
C -> h | λ



In [303]:
production = dict()

In [304]:

for line in codigo_entrada.splitlines():
    left = line.split('->')[0].strip() 
    right = line.split('->')[1].strip()

    print(f"FIRST({left}) = ", end="")
    right
    production[left] = []

    for idx, right_part  in enumerate(right.split("|")): 
 
        production[left].append(right_part.strip())
        if idx > 0:
            print(f" + FIRST({right_part.strip()})", end="")
        else:
            print(f"FIRST({right_part.strip()})", end="")        

    print("")



FIRST(S) = FIRST(ACB) + FIRST(CbB) + FIRST(Ba)
FIRST(A) = FIRST(da) + FIRST(BC)
FIRST(B) = FIRST(g) + FIRST(λ)
FIRST(C) = FIRST(h) + FIRST(λ)


In [305]:
production

{'S': ['ACB', 'CbB', 'Ba'],
 'A': ['da', 'BC'],
 'B': ['g', 'λ'],
 'C': ['h', 'λ']}

In [306]:
## Criar dicionário de first
first_dict = production.copy()
follow_dict = production.copy()
follow_position = production.copy()
for elem in first_dict:
    first_dict[elem] = []
    follow_dict[elem] = []
    follow_position[elem] = []

In [307]:
production

{'S': ['ACB', 'CbB', 'Ba'],
 'A': ['da', 'BC'],
 'B': ['g', 'λ'],
 'C': ['h', 'λ']}

In [308]:
first_dict

{'S': [], 'A': [], 'B': [], 'C': []}

In [309]:
follow_dict

{'S': [], 'A': [], 'B': [], 'C': []}

In [310]:
def merge_lists(elem1, elem2):
    if type(elem1 != list):
        elem1 = list(elem1)
    if type(elem2 != list):
        elem2 = list(elem2)
    return list(set(elem1 + elem2))

In [311]:
def first_set(key):
    ### Checar se já existe no dicionário 

    resultado = []

    if len(key) == 1 and key in variaveis_nao_terminais:
        if (len(first_dict[key]) != 0):
            return first_dict[key]
            
        value = production[key]
    else:
        value = [key]  

    for elem in value:

        if (len(elem) == 1 and elem == key):
            saida = []
        if elem == 'λ':
            saida = 'λ'
        if elem[0] in variaveis_terminais:
            saida = elem[0]
        if elem[0] in variaveis_nao_terminais:
            if (len(first_dict[elem[0]]) != 0 ):
                check = first_dict[elem[0]]
            else:
                check = first_set(elem[0])
            ### CHecar se o first já foi calculado FIRST T
            if 'λ' not in check:
                saida = first_dict[elem[0]]
            else:       
                if(len(elem) > 1):
                    lista_sem_vazio =  [value for value in first_dict[elem[0]] if value != 'λ']
                    
                    if (elem[1:] == variavel_da_vez):
                        saida = lista_sem_vazio
                    else:
                        # saida = lista_sem_vazio.append(first_set(elem[1:]))
                        saida = merge_lists(lista_sem_vazio, first_set(elem[1:]))
                else:
                    saida = first_dict[elem[0]]
        
        # resultado.append(saida)
        resultado = merge_lists(resultado,saida)
        print(resultado)
    first_dict[key] = resultado
    return resultado


In [312]:
for variavel in production:
    variavel_da_vez = variavel
    first_set(variavel)

['d']
['g']
['g', 'λ']
['h']
['h', 'λ']
['g', 'λ', 'd', 'h']
['h', 'g', 'λ']
['g', 'λ', 'd', 'h']
['b']
['λ', 'h', 'g', 'd', 'b']
['a']
['a', 'λ', 'g', 'h', 'd', 'b']


In [313]:
production

{'S': ['ACB', 'CbB', 'Ba'],
 'A': ['da', 'BC'],
 'B': ['g', 'λ'],
 'C': ['h', 'λ']}

In [314]:
### Remover chaves de tamanho maior que um

lista_chaves_desnecessarias = [key for key in first_dict if len(key) > 1 or key in variaveis_terminais]
[first_dict.pop(key) for key in lista_chaves_desnecessarias]


[['h', 'g', 'λ'], ['b'], ['a']]

In [315]:
for key, value in production.items():
    print(key, ' : ', value)

S  :  ['ACB', 'CbB', 'Ba']
A  :  ['da', 'BC']
B  :  ['g', 'λ']
C  :  ['h', 'λ']


In [316]:
elem = 'A'
value = ['aABb','A']
lista_match = [parcela for parcela in value if elem in parcela]
lista_match

['aABb', 'A']

In [317]:

def find_elements_in_prod(production):
    for key, value in production.items():        
        for elem in variaveis_nao_terminais:
            lista_match = [parcela for parcela in value if elem in parcela]
            follow_position[elem] = merge_lists(lista_match,follow_position[elem])  

find_elements_in_prod(production)

In [318]:
for key, value in follow_position.items():
    print(key, ' : ', value)

S  :  []
A  :  ['ACB']
B  :  ['CbB', 'BC', 'ACB', 'Ba']
C  :  ['CbB', 'BC', 'ACB']


In [319]:
for key, value in first_dict.items():
    print(key, ' : ', value)

S  :  ['a', 'λ', 'g', 'h', 'd', 'b']
A  :  ['g', 'λ', 'd', 'h']
B  :  ['g', 'λ']
C  :  ['h', 'λ']


In [320]:
follow_dict

{'S': [], 'A': [], 'B': [], 'C': []}

In [321]:
def find_elem_in_values(elem, prod):
    for key, 

SyntaxError: invalid syntax (<ipython-input-321-e6244399fcad>, line 2)

In [ ]:
def follow_set(key):
    resultado = []
    
    if key == simbolo_inicial:
        resultado = merge_lists(resultado, "$") 
    
    ### Achar na produção, onde existe a chave

    for elem in follow_position[key]:
        print(elem)

    return resultado


follow_set('A')


    
    

aABb


[]